In [1]:
import glob, os
import re

In [2]:
glob_list = []

#loop through subject folders and glob
for subject in range(25):
    glob_list.append(glob.glob(str(subject) + "\\[0-9a-zA-Z]*.csv"))
    
#function to flatten glob
flatten = lambda l: [item for sublist in l for item in sublist]
glob_list = flatten(glob_list)
glob_list

['0\\0_Left.csv',
 '0\\0_Right.csv',
 '0\\10_Left.csv',
 '0\\10_Right.csv',
 '0\\1_Left.csv',
 '0\\1_Right.csv',
 '0\\2_Left.csv',
 '0\\2_Right.csv',
 '0\\3_Left.csv',
 '0\\3_Right.csv',
 '0\\4_Left.csv',
 '0\\4_Right.csv',
 '0\\5_Left.csv',
 '0\\5_Right.csv',
 '0\\6_Left.csv',
 '0\\6_Right.csv',
 '0\\7_Left.csv',
 '0\\7_Right.csv',
 '0\\8_Left.csv',
 '0\\8_Right.csv',
 '0\\9_Left.csv',
 '0\\9_Right.csv',
 '0\\Big_Left.csv',
 '0\\Big_Right.csv',
 '0\\Blue_Left.csv',
 '0\\Blue_Right.csv',
 '0\\Brush_Left.csv',
 '0\\Brush_Right.csv',
 '0\\Bug_Left.csv',
 '0\\Bug_Right.csv',
 '0\\Candy_Left.csv',
 '0\\Candy_Right.csv',
 '0\\CarDrive_Left.csv',
 '0\\CarDrive_Right.csv',
 '0\\Cat_Left.csv',
 '0\\Cat_Right.csv',
 '0\\Cereal_Left.csv',
 '0\\Cereal_Right.csv',
 '0\\Clothes_Left.csv',
 '0\\Clothes_Right.csv',
 '0\\Coat_Left.csv',
 '0\\Coat_Right.csv',
 '0\\Cold_Left.csv',
 '0\\Cold_Right.csv',
 '0\\Come_Left.csv',
 '0\\Come_Right.csv',
 '0\\Cost_Left.csv',
 '0\\Cost_Right.csv',
 '0\\Cry_Left.cs

In [3]:
#make a list of tuples of hand pairings
lr_pairs = list(zip(*[iter(glob_list)]*2))
for i in range(25):
    lr_pairs.remove((str(i)+'\\Blue_Left.csv', str(i)+'\\Blue_Right.csv'))
lr_pairs

[('0\\0_Left.csv', '0\\0_Right.csv'),
 ('0\\10_Left.csv', '0\\10_Right.csv'),
 ('0\\1_Left.csv', '0\\1_Right.csv'),
 ('0\\2_Left.csv', '0\\2_Right.csv'),
 ('0\\3_Left.csv', '0\\3_Right.csv'),
 ('0\\4_Left.csv', '0\\4_Right.csv'),
 ('0\\5_Left.csv', '0\\5_Right.csv'),
 ('0\\6_Left.csv', '0\\6_Right.csv'),
 ('0\\7_Left.csv', '0\\7_Right.csv'),
 ('0\\8_Left.csv', '0\\8_Right.csv'),
 ('0\\9_Left.csv', '0\\9_Right.csv'),
 ('0\\Big_Left.csv', '0\\Big_Right.csv'),
 ('0\\Brush_Left.csv', '0\\Brush_Right.csv'),
 ('0\\Bug_Left.csv', '0\\Bug_Right.csv'),
 ('0\\Candy_Left.csv', '0\\Candy_Right.csv'),
 ('0\\CarDrive_Left.csv', '0\\CarDrive_Right.csv'),
 ('0\\Cat_Left.csv', '0\\Cat_Right.csv'),
 ('0\\Cereal_Left.csv', '0\\Cereal_Right.csv'),
 ('0\\Clothes_Left.csv', '0\\Clothes_Right.csv'),
 ('0\\Coat_Left.csv', '0\\Coat_Right.csv'),
 ('0\\Cold_Left.csv', '0\\Cold_Right.csv'),
 ('0\\Come_Left.csv', '0\\Come_Right.csv'),
 ('0\\Cost_Left.csv', '0\\Cost_Right.csv'),
 ('0\\Cry_Left.csv', '0\\Cry_Right.c

In [4]:
import pandas as pd

df_list = []

for pair in lr_pairs:
    df_left = pd.read_csv(pair[0], index_col=None).drop(['Unnamed: 0'], axis = 1)
    df_right = pd.read_csv(pair[1], index_col=None).drop(['Unnamed: 0'], axis = 1)
    
    #rename columns
    df_left = df_left.add_prefix('left')
    df_right = df_right.add_prefix('right')
    
    #merge
    df = pd.merge(df_left, df_right, left_on='leftTime', right_on='rightTime').drop('rightTime', axis = 1)

    #covert first column to time object
    df['leftTime'] = pd.to_datetime(df['leftTime'].str[:-3], format = '%H:%M:%S.%f')
    
    #difference between rows
    df = df.diff().iloc[1:]
    df['leftTime'] = df['leftTime'].dt.total_seconds()
    
    #add sign and subject using regex of file name
    subject_sign = re.split(r'\\', re.findall('^[^_]+(?=_)', pair[0])[0])
    df.insert(loc = 0, column = 'Subject', value = subject_sign[0])
    df.insert(loc = 0, column = 'Sign', value = subject_sign[1])
    
#this makes velocity columns

#     for col in df.iloc[:,2:]:
#         df[col + ' velocity'] = df[col]/df['Time']
    
    df_list.append(df)

In [5]:
#all signs (except 'Blue') for 25 subjects
len(df_list)

1500

In [6]:
#example entry
df_list[35]

,Sign,Subject,leftTime,left thumbProximal_L_X,left thumbProximal_L_Y,left thumbProximal_L_Z,left thumbDistal_L_X,left thumbDistal_L_Y,left thumbDistal_L_Z,left thumbEF_L_X,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,Hungry,0,0.033,-0.003069,-0.005275,0.005377,-0.003816,-0.004857,0.005608,-0.004414,...,0.004107,-0.000659,-0.002197,0.003803,-0.000530,0.001926,-0.000143,0.000351,0.001219,-0.000448
2,Hungry,0,0.026,-0.001764,-0.004115,0.001626,-0.001930,-0.003919,0.001489,-0.002002,...,0.001137,-0.000269,-0.000394,0.001037,0.000620,0.001447,-0.000302,0.001102,0.001161,-0.000402
3,Hungry,0,0.029,-0.001353,-0.002294,0.002112,-0.001203,-0.001881,0.001159,-0.001035,...,0.001369,-0.000648,-0.000467,0.001406,0.000575,0.000405,0.001136,0.001011,0.000304,0.001173
4,Hungry,0,0.035,-0.002852,0.002370,-0.002223,-0.002433,0.002183,-0.002436,-0.001816,...,0.000532,0.000103,0.000654,0.000618,0.002438,0.000376,0.001927,0.002455,0.000469,0.002012
5,Hungry,0,0.032,-0.002808,0.001876,-0.002413,-0.002978,0.001908,-0.002239,-0.002983,...,0.000638,-0.000744,0.000458,0.000887,0.000344,-0.000738,0.002516,0.000463,-0.000483,0.002765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,Hungry,0,0.032,-0.000166,0.000071,-0.000069,-0.000172,-0.000106,0.000143,-0.000245,...,0.000532,-0.005505,-0.002449,-0.000298,-0.003224,-0.002921,0.002098,-0.004334,-0.002862,0.001267
757,Hungry,0,0.033,-0.000562,0.000314,-0.000336,-0.000559,0.000229,-0.000243,-0.000552,...,0.001466,-0.006570,-0.003300,0.000742,-0.003565,-0.004649,0.004747,-0.004666,-0.004443,0.004024
758,Hungry,0,0.035,-0.000995,0.000647,-0.000363,-0.001169,0.000992,-0.000595,-0.001157,...,0.000672,-0.006849,-0.002689,0.000387,-0.003166,-0.002175,0.003335,-0.003353,-0.002255,0.003049
759,Hungry,0,0.377,-0.000041,0.000389,-0.002611,0.000293,0.001245,-0.004029,0.000969,...,0.001266,-0.052832,-0.009714,-0.006717,0.004473,0.040545,0.023070,0.007423,0.034512,0.015088


In [7]:
#concatenate list of frames
sign_frame = pd.concat(df_list, axis=0, ignore_index=False)
sign_frame

,Sign,Subject,leftTime,left thumbProximal_L_X,left thumbProximal_L_Y,left thumbProximal_L_Z,left thumbDistal_L_X,left thumbDistal_L_Y,left thumbDistal_L_Z,left thumbEF_L_X,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,0,0,0.028,0.000388,0.000133,0.000050,0.000406,0.000153,0.000063,0.000403,...,-0.000321,-0.000166,0.000114,-0.000298,-8.360000e-05,-0.000393,-0.000142,-0.000071,-0.000331,-0.000119
2,0,0,0.036,0.000420,0.000077,-0.000248,0.000443,0.000085,-0.000239,0.000429,...,-0.000235,-0.000095,0.000121,-0.000193,4.000000e-05,-0.000276,-0.000075,0.000035,-0.000261,-0.000034
3,0,0,0.033,0.000223,0.000264,-0.000160,0.000164,0.000275,-0.000170,0.000121,...,-0.000251,-0.000084,0.000163,-0.000204,-1.250000e-04,-0.000481,0.000008,-0.000147,-0.000522,0.000054
4,0,0,0.034,0.000103,0.000078,-0.000318,0.000105,0.000103,-0.000307,0.000054,...,-0.000213,-0.000179,0.000077,-0.000056,1.000000e-07,-0.000630,0.000164,-0.000104,-0.000869,0.000321
5,0,0,0.036,0.000055,0.000233,-0.000030,0.000002,0.000263,-0.000031,-0.000022,...,-0.000001,0.000016,0.000356,0.000150,5.750000e-04,0.000152,0.000179,0.000504,0.000025,0.000330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,Yellow,24,0.041,-0.000081,0.000077,-0.000299,-0.000051,-0.000085,-0.000311,-0.000089,...,0.000739,-0.003499,-0.001501,-0.000228,-2.789460e-02,-0.018280,0.000122,-0.026445,-0.018375,-0.000844
840,Yellow,24,0.040,-0.000211,0.000190,-0.000301,-0.000174,0.000163,-0.000348,-0.000167,...,-0.002417,-0.001886,-0.000467,-0.002440,-6.380100e-03,0.002420,-0.007261,-0.008172,-0.000481,-0.007284
841,Yellow,24,0.041,-0.000124,0.000341,-0.000163,-0.000145,0.000336,-0.000134,-0.000140,...,-0.001664,0.001196,0.001054,-0.001449,9.235200e-03,0.008616,-0.003942,0.007797,0.007248,-0.003728
842,Yellow,24,0.034,-0.000187,0.000359,-0.000166,-0.000138,0.000345,-0.000232,-0.000073,...,-0.000334,0.000911,0.000960,-0.000300,7.136000e-04,0.000893,-0.000346,0.000451,0.000660,-0.000311


In [8]:
agg_df = sign_frame.groupby(['Subject','Sign']).agg(['mean','std'])
agg_df = agg_df.reset_index()
agg_df.head()

Subject Sign  leftTime           left thumbProximal_L_X            \
                    mean       std                   mean       std   
0       0    0  0.043573  0.007938           9.093222e-06  0.000329   
1       0    1  0.041300  0.007824           5.417744e-06  0.000294   
2       0   10  0.065428  0.018792           1.764326e-05  0.000718   
3       0    2  0.041766  0.007072          -3.434540e-07  0.000270   
4       0    3  0.043742  0.008602          -1.805375e-05  0.000445   

  left thumbProximal_L_Y           left thumbProximal_L_Z            ...  \
                    mean       std                   mean       std  ...   
0               0.000013  0.000287              -0.000004  0.000213  ...   
1               0.000025  0.000349              -0.000021  0.000199  ...   
2               0.000009  0.000812              -0.000047  0.000664  ...   
3               0.000022  0.000360              -0.000014  0.000226  ...   
4               0.000025  0.000454              -0.000018  0.000302  ...   

  right armBackLateral_L_Y           right armBackLateral_L_Z            \
                      mean       std                     mean       std   
0            -5.686067e-06  0.004467                 0.000011  0.002841   
1            -2.150715e-05  0.005544                 0.000045  0.004783   
2            -4.001783e-05  0.012821                 0.000206  0.012200   
3            -2.253719e-05  0.006004                 0.000005  0.005082   
4             6.469388e-07  0.004286                 0.000004  0.005211   

  right armBackMedial_L_X           right armBackMedial_L_Y            \
                     mean       std                    mean       std   
0               -0.000033  0.004745               -0.000039  0.004753   
1               -0.000033  0.005855               -0.000014  0.006204   
2               -0.000091  0.012339               -0.000015  0.011673   
3               -0.000006  0.006624               -0.000020  0.006283   
4               -0.000006  0.005767               -0.000001  0.004373   

  right armBackMedial_L_Z            
                     mean       std  
0                0.000025  0.003106  
1                0.000007  0.004511  
2                0.000116  0.007522  
3                0.000006  0.004975  
4                0.000007  0.004265  

[5 rows x 328 columns]

In [9]:
agg_df.shape

(1500, 328)

In [10]:
agg_df = agg_df.dropna(how='any')

In [11]:
# agg_df.isnull().sum()

In [12]:
#Separating features and label
X = agg_df.iloc[:, 2:328]
y = agg_df.iloc[:, 1]

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [14]:
#Feature scaling

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Method 1: Using LDA and Random Forest**

In [15]:
#Performing LDA

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=60)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

c:\users\chenson\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [16]:
#Train a Random Forest Model

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=2, random_state=0)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

c:\users\chenson\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [17]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy = ' + str(accuracy_score(y_test, y_pred)))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 4 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Accuracy = 0.19


**Method 2: Using LDA only**

In [18]:
import numpy as np

clf = LDA()
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.8

# Confusion

In [19]:
confusion = pd.DataFrame(zip(clf.predict(X_test), y_test), columns=['actual', 'predicted'])

## confirm accuracy

In [20]:
confusion['compare'] = (confusion.actual == confusion.predicted)
confusion

,actual,predicted,compare
0,Warm,Warm,True
1,8,8,True
2,Yellow,Yellow,True
3,When,When,True
4,Good,Good,True
...,...,...,...
295,Neutral,Neutral,True
296,Small,Small,True
297,Green,Green,True
298,Why,Why,True


In [21]:
confusion.compare.sum()/len(confusion)

0.8

## Which signs have errors??

In [22]:
misclass = confusion[confusion.compare == False]

In [23]:
misclass.groupby(['actual', 'predicted'])[['predicted']].agg('count')

predicted
actual   predicted           
0        Deaf               1
         Dog                1
         Orange             1
1        3                  1
         Dog                1
         Green              1
2        3                  2
3        2                  1
4        5                  2
5        4                  2
7        Yellow             1
8        Yellow             1
9        8                  1
Big      With               2
Brush    Where              1
Bug      Cat                1
CarDrive Big                1
Cat      Mom                1
Cereal   Brush              1
         Candy              1
Coat     With               1
Cost     What               2
Deaf     Cereal             2
         Cry                1
         Orange             1
Dog      3                  1
Drink    Orange             1
Go       Come               1
         Socks              1
Good     Happy              1
         Stop               2
Green    Where              1
Happy    Clothes            1
         Finish             1
Hungry   Please             1
Neutral  10                 1
         Big                1
         Cold               1
         Go                 1
         More               1
         Red                1
         Socks              2
         When               1
         With               1
Red      1                  1
         Why                1
Small    Stop               1
Stop     Cost               1
         What               1
What     Cost               3

## Crosstab

In [24]:
pd.crosstab(misclass.actual, misclass.predicted)

predicted,1,10,2,3,4,5,8,Big,Brush,Candy,...,Please,Red,Socks,Stop,What,When,Where,Why,With,Yellow
actual,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
